# The battle of neighborhood (Week 2)

# Intro

New York City's demographics show that it is a large and ethnically diverse metropolis. It is the largest city in the United States with a long history of international immigration. New York City was home to nearly 8.5 million people in 2014, accounting for over 40% of the population of New York State and a slightly lower percentage of the New York metropolitan area, home to approximately 23.6 million. Over the last decade the city has been growing faster than the region. The New York region continues to be by far the leading metropolitan gateway for legal immigrants admitted into the United States.

Throughout its history, New York City has been a major point of entry for immigrants; the term "melting pot" was coined to describe densely populated immigrant neighborhoods on the Lower East Side. As many as 800 languages are spoken in New York, making it the most linguistically diverse city in the world. English remains the most widely spoken language, although there are areas in the outer boroughs in which up to 25% of people speak English as an alternate language, and/or have limited or no English language fluency. English is least spoken in neighborhoods such as Flushing, Sunset Park, and Corona.

With it's diverse culture , comes diverse food items. There are many resturants in New york City, each beloning to different categories like Chinese , Indian , French etc.

So as part of this project , we will list and visualize all major parts of New York City that has great indian resturants.

## Data

Data For this project we need the following data :

New York City data that contains list Boroughs, Neighborhoods along with their latitude and longitude. Data source : https://cocl.us/new_york_dataset Description : This data set contains the required information. And we will use this data set to explore various neighborhoods of new york city. Indian resturants in each neighborhood of new york city. Data source : Fousquare API Description : By using this api we will get all the venues in each neighborhood. We can filter these venues to get only indian resturants. GeoSpace data Data source : https://data.cityofnewyork.us/City-Government/Borough-Boundaries/tqmj-j8zm Description : By using this geo space data we will get the New york Borough boundaries that will help us visualize choropleth map.

## Import libraries

In [98]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import urllib.request
import json
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
%matplotlib inline
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Libraries imported.')

Libraries imported.


In [54]:
CLIENT_ID = 'XA4L4ESICRTSGS5GINHBFROT2VMUXSDM0OIM3WBGZFBJ5215' # your Foursquare ID
CLIENT_SECRET = 'LDGFKKQCGQXK5D4E4BOEYLDBL54ZOXCYAYCYWZBQQL0BY5CB' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XA4L4ESICRTSGS5GINHBFROT2VMUXSDM0OIM3WBGZFBJ5215
CLIENT_SECRET:LDGFKKQCGQXK5D4E4BOEYLDBL54ZOXCYAYCYWZBQQL0BY5CB


In [55]:
def geo_location(address):
    # get geo location of address
    geolocator = Nominatim(user_agent="foursquare_agent")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    return latitude,longitude


def get_venues(lat,lng):
    #set variables
    radius=400
    LIMIT=100
    #url to fetch data from foursquare api
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
    # get all the data
    results = requests.get(url).json()
    venue_data=results["response"]['groups'][0]['items']
    venue_details=[]
    for row in venue_data:
        try:
            venue_id=row['venue']['id']
            venue_name=row['venue']['name']
            venue_category=row['venue']['categories'][0]['name']
            venue_details.append([venue_id,venue_name,venue_category])
        except KeyError:
            pass
    column_names=['ID','Name','Category']
    df = pd.DataFrame(venue_details,columns=column_names)
    return df


def get_venue_details(venue_id):
    #url to fetch data from foursquare api
    url = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(
            venue_id,
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION)
    # get all the data
    results = requests.get(url).json()
    print(results)
    venue_data=results['response']['venue']
    venue_details=[]
    try:
        venue_id=venue_data['id']
        venue_name=venue_data['name']
        venue_likes=venue_data['likes']['count']
        venue_rating=venue_data['rating']
        venue_tips=venue_data['tips']['count']
        venue_details.append([venue_id,venue_name,venue_likes,venue_rating,venue_tips])
    except KeyError:
        pass
    column_names=['ID','Name','Likes','Rating','Tips']
    df = pd.DataFrame(venue_details,columns=column_names)
    return df


def get_new_york_data():
    url='https://cocl.us/new_york_dataset'
    resp=requests.get(url).json()
    # all data is present in features label
    features=resp['features']
    # define the dataframe columns
    column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
    # instantiate the dataframe
    new_york_data = pd.DataFrame(columns=column_names)
    for data in features:
        borough = data['properties']['borough'] 
        neighborhood_name = data['properties']['name']
        neighborhood_latlon = data['geometry']['coordinates']
        neighborhood_lat = neighborhood_latlon[1]
        neighborhood_lon = neighborhood_latlon[0]
        new_york_data = new_york_data.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    return new_york_data

In [56]:
ny_data = get_new_york_data()
ny_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [57]:
ny_data.shape

(306, 4)

In [82]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [83]:
NY_data = ny_data.reset_index(drop=True)
NY_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [84]:
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(NY_data['Latitude'], NY_data['Longitude'], NY_data['Borough'], NY_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

## Foursquare venues

In [85]:
import urllib
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            # make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            # return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',  
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [86]:
LIMIT = 500 
radius = 5000 
CLIENT_ID = 'XA4L4ESICRTSGS5GINHBFROT2VMUXSDM0OIM3WBGZFBJ5215' # your Foursquare ID
CLIENT_SECRET = 'LDGFKKQCGQXK5D4E4BOEYLDBL54ZOXCYAYCYWZBQQL0BY5CB' # your Foursquare Secret
VERSION = '20181020'

In [88]:
#neighborhoods = neighborhoods.reset_index(drop=True)
newyork_venues_cantonese = getNearbyVenues(names=ny_data['Neighborhood'], latitudes=ny_data['Latitude'], longitudes=ny_data['Longitude'], radius=1000, categoryIds='52af3a7c3cf9994f4e043bed')
newyork_venues_cantonese.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bay Ridge,40.625801,-74.030621,Szechuan Delight,40.618908,-74.021592,Cantonese Restaurant
1,Bensonhurst,40.611009,-73.995180,香港地New Forest Cafe,40.614453,-74.000350,Cantonese Restaurant
2,Bensonhurst,40.611009,-73.995180,Siao Chao Wong,40.603695,-73.997322,Cantonese Restaurant
3,Bensonhurst,40.611009,-73.995180,大華小炒皇,40.614777,-73.994774,Cantonese Restaurant
4,Bensonhurst,40.611009,-73.995180,滋味粤菜館,40.608757,-74.004974,Cantonese Restaurant


In [89]:
newyork_venues_cantonese.shape

(245, 7)

In [90]:
def addToMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['Neighborhood'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

In [91]:
map_newyork_cantonese = folium.Map(location=[latitude, longitude], zoom_start=10)
addToMap(newyork_venues_cantonese, 'red', map_newyork_cantonese)

map_newyork_cantonese

In [92]:
def addColumn(startDf, columnTitle, dataDf):
    grouped = dataDf.groupby('Neighborhood').count()
    
    for n in startDf['Neighborhood']:
        try:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = grouped.loc[n, 'Venue']
        except:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = 0

In [93]:
NY_grouped = newyork_venues_cantonese.groupby('Neighborhood').count()
NY_grouped
#print('There are {} uniques categories.'.format(len(newyork_venues_cantonese['Venue Category'].unique())))

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Auburndale,2,2,2,2,2,2
Bath Beach,14,14,14,14,14,14
Bay Ridge,1,1,1,1,1,1
Bensonhurst,14,14,14,14,14,14
Chelsea,1,1,1,1,1,1
Chinatown,25,25,25,25,25,25
Civic Center,23,23,23,23,23,23
Clinton,1,1,1,1,1,1
Douglaston,2,2,2,2,2,2


## Analyzing each neighborhood

In [94]:
# one hot encoding
NY_onehot = pd.get_dummies(newyork_venues_cantonese[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
NY_onehot['Neighborhood'] = newyork_venues_cantonese['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [NY_onehot.columns[-1]] + list(NY_onehot.columns[:-1])
NY_onehot = NY_onehot[fixed_columns]

NY_onehot.head()

,Neighborhood,Asian Restaurant,Cantonese Restaurant,Cha Chaan Teng,Chinese Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Noodle House,Seafood Restaurant
0,Bay Ridge,0,1,0,0,0,0,0,0
1,Bensonhurst,0,1,0,0,0,0,0,0
2,Bensonhurst,0,1,0,0,0,0,0,0
3,Bensonhurst,0,1,0,0,0,0,0,0
4,Bensonhurst,0,1,0,0,0,0,0,0


In [95]:
NY_grouped = NY_onehot.groupby('Neighborhood').mean().reset_index()
NY_grouped

,Neighborhood,Asian Restaurant,Cantonese Restaurant,Cha Chaan Teng,Chinese Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Noodle House,Seafood Restaurant
0,Auburndale,0.000000,0.500000,0.000000,0.500000,0.000000,0.0000,0.000000,0.0000
1,Bath Beach,0.000000,1.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000
2,Bay Ridge,0.000000,1.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000
3,Bensonhurst,0.000000,1.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000
4,Chelsea,0.000000,1.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000
5,Chinatown,0.000000,0.680000,0.040000,0.200000,0.080000,0.0000,0.000000,0.0000
6,Civic Center,0.000000,0.652174,0.043478,0.217391,0.086957,0.0000,0.000000,0.0000
7,Clinton,0.000000,1.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000
8,Douglaston,0.000000,1.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000
9,Dumbo,0.000000,1.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000


In [96]:
NY_grouped = NY_onehot.groupby('Neighborhood').mean().reset_index()
NY_grouped

,Neighborhood,Asian Restaurant,Cantonese Restaurant,Cha Chaan Teng,Chinese Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Noodle House,Seafood Restaurant
0,Auburndale,0.000000,0.500000,0.000000,0.500000,0.000000,0.0000,0.000000,0.0000
1,Bath Beach,0.000000,1.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000
2,Bay Ridge,0.000000,1.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000
3,Bensonhurst,0.000000,1.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000
4,Chelsea,0.000000,1.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000
5,Chinatown,0.000000,0.680000,0.040000,0.200000,0.080000,0.0000,0.000000,0.0000
6,Civic Center,0.000000,0.652174,0.043478,0.217391,0.086957,0.0000,0.000000,0.0000
7,Clinton,0.000000,1.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000
8,Douglaston,0.000000,1.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000
9,Dumbo,0.000000,1.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000


In [108]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [111]:
num_top_venues = 8

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = NY_grouped['Neighborhood']

for ind in np.arange(NY_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(NY_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue
0,Auburndale,Cantonese Restaurant,Chinese Restaurant,Asian Restaurant,Cha Chaan Teng,Dim Sum Restaurant,Dumpling Restaurant,Noodle House,Seafood Restaurant
1,Bath Beach,Cantonese Restaurant,Asian Restaurant,Cha Chaan Teng,Chinese Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Noodle House,Seafood Restaurant
2,Bay Ridge,Cantonese Restaurant,Asian Restaurant,Cha Chaan Teng,Chinese Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Noodle House,Seafood Restaurant
3,Bensonhurst,Cantonese Restaurant,Asian Restaurant,Cha Chaan Teng,Chinese Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Noodle House,Seafood Restaurant
4,Chelsea,Cantonese Restaurant,Asian Restaurant,Cha Chaan Teng,Chinese Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Noodle House,Seafood Restaurant


## Clustering neighborhoods

In [112]:
# set number of clusters
kclusters = 4

NY_grouped_clustering = NY_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(NY_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:8]

array([3, 1, 1, 1, 1, 3, 3, 1], dtype=int32)

In [113]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted
NY_merged = NY_data
NY_merged = NY_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

In [114]:
NY_merged.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)
NY_merged.reset_index(inplace = True, drop = True)
#NY_merged['Cluster Labels'].astype(int)
pd.to_numeric(NY_merged['Cluster Labels'], downcast='integer')
NY_merged

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue
0,Brooklyn,Bay Ridge,40.625801,-74.030621,1.0,Cantonese Restaurant,Asian Restaurant,Cha Chaan Teng,Chinese Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Noodle House,Seafood Restaurant
1,Brooklyn,Bensonhurst,40.611009,-73.995180,1.0,Cantonese Restaurant,Asian Restaurant,Cha Chaan Teng,Chinese Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Noodle House,Seafood Restaurant
2,Brooklyn,Sunset Park,40.645103,-74.010316,1.0,Cantonese Restaurant,Dim Sum Restaurant,Asian Restaurant,Cha Chaan Teng,Chinese Restaurant,Dumpling Restaurant,Noodle House,Seafood Restaurant
3,Brooklyn,Gravesend,40.595260,-73.973471,1.0,Cantonese Restaurant,Asian Restaurant,Cha Chaan Teng,Chinese Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Noodle House,Seafood Restaurant
4,Brooklyn,Williamsburg,40.707144,-73.958115,0.0,Dim Sum Restaurant,Asian Restaurant,Cantonese Restaurant,Cha Chaan Teng,Chinese Restaurant,Dumpling Restaurant,Noodle House,Seafood Restaurant
5,Brooklyn,Bath Beach,40.599519,-73.998752,1.0,Cantonese Restaurant,Asian Restaurant,Cha Chaan Teng,Chinese Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Noodle House,Seafood Restaurant
6,Brooklyn,Dyker Heights,40.619219,-74.019314,1.0,Cantonese Restaurant,Asian Restaurant,Cha Chaan Teng,Chinese Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Noodle House,Seafood Restaurant
7,Brooklyn,North Side,40.714823,-73.958809,0.0,Dim Sum Restaurant,Asian Restaurant,Cantonese Restaurant,Cha Chaan Teng,Chinese Restaurant,Dumpling Restaurant,Noodle House,Seafood Restaurant
8,Brooklyn,South Side,40.710861,-73.958001,0.0,Dim Sum Restaurant,Asian Restaurant,Cantonese Restaurant,Cha Chaan Teng,Chinese Restaurant,Dumpling Restaurant,Noodle House,Seafood Restaurant
9,Brooklyn,Ocean Parkway,40.613060,-73.968367,2.0,Chinese Restaurant,Asian Restaurant,Cantonese Restaurant,Cha Chaan Teng,Dim Sum Restaurant,Dumpling Restaurant,Noodle House,Seafood Restaurant


In [115]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(NY_merged['Latitude'], NY_merged['Longitude'], NY_merged['Neighborhood'], NY_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [116]:
NY_merged.loc[NY_merged['Cluster Labels'] == 0, NY_merged.columns[[1] + list(range(5, NY_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue
4,Williamsburg,Dim Sum Restaurant,Asian Restaurant,Cantonese Restaurant,Cha Chaan Teng,Chinese Restaurant,Dumpling Restaurant,Noodle House,Seafood Restaurant
7,North Side,Dim Sum Restaurant,Asian Restaurant,Cantonese Restaurant,Cha Chaan Teng,Chinese Restaurant,Dumpling Restaurant,Noodle House,Seafood Restaurant
8,South Side,Dim Sum Restaurant,Asian Restaurant,Cantonese Restaurant,Cha Chaan Teng,Chinese Restaurant,Dumpling Restaurant,Noodle House,Seafood Restaurant
16,Midtown,Dim Sum Restaurant,Asian Restaurant,Cantonese Restaurant,Cha Chaan Teng,Chinese Restaurant,Dumpling Restaurant,Noodle House,Seafood Restaurant
46,Sutton Place,Dim Sum Restaurant,Asian Restaurant,Cantonese Restaurant,Cha Chaan Teng,Chinese Restaurant,Dumpling Restaurant,Noodle House,Seafood Restaurant


In [117]:
NY_merged.loc[NY_merged['Cluster Labels'] == 1, NY_merged.columns[[1] + list(range(5, NY_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue
0,Bay Ridge,Cantonese Restaurant,Asian Restaurant,Cha Chaan Teng,Chinese Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Noodle House,Seafood Restaurant
1,Bensonhurst,Cantonese Restaurant,Asian Restaurant,Cha Chaan Teng,Chinese Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Noodle House,Seafood Restaurant
2,Sunset Park,Cantonese Restaurant,Dim Sum Restaurant,Asian Restaurant,Cha Chaan Teng,Chinese Restaurant,Dumpling Restaurant,Noodle House,Seafood Restaurant
3,Gravesend,Cantonese Restaurant,Asian Restaurant,Cha Chaan Teng,Chinese Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Noodle House,Seafood Restaurant
5,Bath Beach,Cantonese Restaurant,Asian Restaurant,Cha Chaan Teng,Chinese Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Noodle House,Seafood Restaurant
6,Dyker Heights,Cantonese Restaurant,Asian Restaurant,Cha Chaan Teng,Chinese Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Noodle House,Seafood Restaurant
10,Fort Hamilton,Cantonese Restaurant,Asian Restaurant,Cha Chaan Teng,Chinese Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Noodle House,Seafood Restaurant
12,Upper East Side,Cantonese Restaurant,Asian Restaurant,Cha Chaan Teng,Chinese Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Noodle House,Seafood Restaurant
13,Yorkville,Cantonese Restaurant,Asian Restaurant,Cha Chaan Teng,Chinese Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Noodle House,Seafood Restaurant
14,Lenox Hill,Cantonese Restaurant,Asian Restaurant,Cha Chaan Teng,Chinese Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Noodle House,Seafood Restaurant


In [118]:
NY_merged.loc[NY_merged['Cluster Labels'] == 2, NY_merged.columns[[1] + list(range(5, NY_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue
9,Ocean Parkway,Chinese Restaurant,Asian Restaurant,Cantonese Restaurant,Cha Chaan Teng,Dim Sum Restaurant,Dumpling Restaurant,Noodle House,Seafood Restaurant
19,East Village,Chinese Restaurant,Asian Restaurant,Cantonese Restaurant,Cha Chaan Teng,Dim Sum Restaurant,Dumpling Restaurant,Noodle House,Seafood Restaurant
24,West Village,Chinese Restaurant,Asian Restaurant,Cantonese Restaurant,Cha Chaan Teng,Dim Sum Restaurant,Dumpling Restaurant,Noodle House,Seafood Restaurant
25,Gramercy,Chinese Restaurant,Asian Restaurant,Cantonese Restaurant,Cha Chaan Teng,Dim Sum Restaurant,Dumpling Restaurant,Noodle House,Seafood Restaurant


In [119]:
NY_merged.loc[NY_merged['Cluster Labels'] == 3, NY_merged.columns[[1] + list(range(5, NY_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue
11,Chinatown,Cantonese Restaurant,Chinese Restaurant,Dim Sum Restaurant,Cha Chaan Teng,Asian Restaurant,Dumpling Restaurant,Noodle House,Seafood Restaurant
18,Greenwich Village,Cantonese Restaurant,Chinese Restaurant,Asian Restaurant,Cha Chaan Teng,Dim Sum Restaurant,Dumpling Restaurant,Noodle House,Seafood Restaurant
20,Lower East Side,Cantonese Restaurant,Chinese Restaurant,Asian Restaurant,Cha Chaan Teng,Dim Sum Restaurant,Dumpling Restaurant,Noodle House,Seafood Restaurant
21,Tribeca,Cantonese Restaurant,Chinese Restaurant,Cha Chaan Teng,Asian Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Noodle House,Seafood Restaurant
22,Little Italy,Cantonese Restaurant,Chinese Restaurant,Dim Sum Restaurant,Cha Chaan Teng,Asian Restaurant,Dumpling Restaurant,Noodle House,Seafood Restaurant
23,Soho,Cantonese Restaurant,Chinese Restaurant,Dim Sum Restaurant,Cha Chaan Teng,Asian Restaurant,Dumpling Restaurant,Noodle House,Seafood Restaurant
26,Jackson Heights,Cantonese Restaurant,Chinese Restaurant,Asian Restaurant,Cha Chaan Teng,Dim Sum Restaurant,Dumpling Restaurant,Noodle House,Seafood Restaurant
27,Elmhurst,Cantonese Restaurant,Chinese Restaurant,Asian Restaurant,Cha Chaan Teng,Dim Sum Restaurant,Dumpling Restaurant,Noodle House,Seafood Restaurant
30,Auburndale,Cantonese Restaurant,Chinese Restaurant,Asian Restaurant,Cha Chaan Teng,Dim Sum Restaurant,Dumpling Restaurant,Noodle House,Seafood Restaurant
40,Noho,Cantonese Restaurant,Chinese Restaurant,Asian Restaurant,Cha Chaan Teng,Dim Sum Restaurant,Dumpling Restaurant,Noodle House,Seafood Restaurant
